<a href="https://colab.research.google.com/github/gzquse/GRADIENT_IMAGE/blob/main/Bell_CUDA_Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -v --no-cache-dir cuquantum-python

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Running command python setup.py egg_info
  [cuquantum-python] Trying to detect CUDA version from libraries: ['libnvrtc.so.12', 'libnvrtc.so.11.2', 'libnvrtc.so.11.1', 'libnvrtc.so.11.0']
  [cuquantum-python] Looking for library: libnvrtc.so.12
  [cuquantum-python] Detected version: 12020
  [cuquantum-python] Installing cuquantum-python-cu12...
  /usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!

          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************

  !!
    dist.fetch_build_eggs(dist.setup_requires)
  /usr/local/lib/python3.10/dist-packages/s

In [4]:
!pip install -v --no-cache-dir qiskit-aer-gpu

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 139.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.1/895.1 kB 347.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 215.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 MB 271.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 MB 180.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 MB 155.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 329.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 206.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 264.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 288.4 MB/s eta 0:00:00
     ━━━━━━━━

In [7]:
import numpy as np
import cupy as cp

import cuquantum
from cuquantum import custatevec as cusv


nIndexBits = 3
nSvSize    = (1 << nIndexBits)
nMaxShots  = 5
nShots     = 5

bitStringLen  = 2;
bitOrdering   = np.asarray([0, 1], dtype=np.int32)

bitStrings = np.empty((nShots,), dtype=np.int64)
bitStrings_expected = np.asarray([0b00, 0b01, 0b10, 0b11, 0b11], dtype=np.int64)

h_sv          = np.asarray([0.0+0.0j, 0.0+0.1j, 0.1+0.1j, 0.1+0.2j,
                            0.2+0.2j, 0.3+0.3j, 0.3+0.4j, 0.4+0.5j], dtype=np.complex64)

d_sv = cp.asarray(h_sv)

# In real appliction, random numbers in range [0, 1) will be used.
randnums      = np.asarray([0.1, 0.8, 0.4, 0.6, 0.2], dtype=np.float64)

########################################################################

# cuStateVec handle initialization
handle = cusv.create()

# create sampler and check the size of external workspace
sampler, extraWorkspaceSizeInBytes = cusv.sampler_create(
    handle, d_sv.data.ptr, cuquantum.cudaDataType.CUDA_C_32F, nIndexBits, nMaxShots)

# allocate external workspace
extraWorkspace = cp.cuda.alloc(extraWorkspaceSizeInBytes)

# sample preprocess
cusv.sampler_preprocess(
    handle, sampler, extraWorkspace.ptr, extraWorkspaceSizeInBytes)

# sample bit strings
cusv.sampler_sample(
    handle, sampler, bitStrings.ctypes.data, bitOrdering.ctypes.data, bitStringLen,
    randnums.ctypes.data, nShots, cusv.SamplerOutput.ASCENDING_ORDER)

# destroy sampler
cusv.sampler_destroy(sampler)

# destroy handle
cusv.destroy(handle)

if not np.allclose(bitStrings, bitStrings_expected):
    raise ValueError("results mismatch")
print("test passed")

test passed
